In [ ]:
import cv2 
import numpy as np 
import os
from matplotlib import pyplot as plt 
import mediapipe as mp 
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import pickle 
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import numpy as np
from sklearn.metrics import accuracy_score
from hmmlearn import hmm


In [ ]:
dir_path = "../DataSet" 
actions = np.array(os.listdir(dir_path))


results_path = os.path.join('data_results')



for action in actions: 
    num_of_videos = os.listdir(dir_path+'/'+action)
    for video in range(len(num_of_videos)): 
        try: 
            os.makedirs(os.path.join(results_path, action, str(video)))
        except: 
            pass 


In [ ]:

label_map = {label: num for num, label in enumerate(actions)}

In [ ]:
print(label_map)

In [ ]:

missing_files = []
sequences, labels = [], []

for action in actions: 
    file_list = os.listdir(dir_path+"/"+action)

    for video in range(len(file_list)): 
        window = []
        filename = file_list[video]
        cap = cv2.VideoCapture(dir_path+"/"+action+"/"+filename)
        num_of_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        for frame_num in range(num_of_frames): 
            try: 
                res = np.load(os.path.join(results_path, action, str(video), "{}.npy".format(frame_num)))
                print(res)
                window.append(res)
                print("window")
                print(window)
            except FileNotFoundError: 
                missing_files.append(action + '/' + str(video)+ '/' +str(frame_num))
                print("Missing ")
                print(missing_files)
                continue
        sequences.append(window)
        labels.append(action)

In [ ]:
X = np.array(sequences)
Y= np.array(labels)

In [ ]:


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42 )

In [ ]:
y_train_encoded = np.array([label_map[label] for label in y_train])
y_test_encoded = np.array([label_map[label] for label in y_test])

## Hyper Parameter tuning 



In [ ]:
import optuna 
from sklearn.metrics import accuracy_score

def objective(trial): 
    n_components = trial.suggest_int('n_components',1,5, step=1)
    covariance_type = trial.suggest_categorical('covariance_type', ['spherical', 'diag', 'tied','full'])
    models = [hmm.GaussianHMM(n_components=n_components, covariance_type= covariance_type, n_iter=1000) for _ in range(n_classes)]

    for gesture_class in range(n_classes): 
        gesture_sequences = X_train[y_train_encoded  == gesture_class]
        lengths = [len(seq) for seq in gesture_sequences]
        concatenated_sequences = np.concatenate(gesture_sequences)
        models[gesture_class].fit(concatenated_sequences, lengths)
    
    y_pred = []
    for sequence in X_test: 
        likelihoods = [model.score(sequence) for model in models]
        y_pred.append(np.argmax(likelihoods))

    accuracy = accuracy_score (y_test_encoded, y_pred)
    print(accuracy)

    return accuracy

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=40)
best_trial = study.best_trial 
print(f"Best trial:{best_trial.number}, val_accurracy: {best_trial.value}, params: {best_trial.params}")

## Training the model


In [ ]:
best_params = study.best_params

n_components = best_params["n_components"]
covariance_type = best_params["covariance_type"]
print(covariance_type)
print(n_components)
models =[hmm.GaussianHMM(n_components=3, covariance_type='tied', n_iter=1000) for _ in range(n_classes)] 

for gesture_class in range(n_classes): 
    gesture_sequences = X_train[y_train_encoded  == gesture_class]
    lengths = [len(seq) for seq in gesture_sequences]
    concatenated_sequences = np.concatenate(gesture_sequences)
    models[gesture_class].fit(concatenated_sequences, lengths)
    
y_pred = []
for sequence in X_test: 
    likelihoods = [model.score(sequence) for model in models]
    y_pred.append(np.argmax(likelihoods))

accuracy = accuracy_score (y_test_encoded, y_pred)
print(f"Classification accuracy with the best hyperparameters: {accuracy}")



In [ ]:

accuracy = accuracy_score(y_test_encoded, y_pred)
print(f"Classification accuracy: {accuracy}")

In [ ]:

print(classification_report(y_test_encoded,y_pred))
cm = confusion_matrix(y_test_encoded, y_pred)
print(cm)

In [ ]:

class_names = ['Blue', 'Family', 'Happy', 'Man']

plt.figure(figsize=(10,7))

sns.heatmap(cm, annot=True, fmt='d', cmap='Oranges', xticklabels=class_names, yticklabels=class_names)

plt.title('Confusion Matrix')

plt.ylabel('True label')

plt.xlabel('Predicted label')

plt.show()

In [ ]:

with open("hmm_models_66_percent", "wb") as f: 
    pickle.dump(models,f)

In [ ]:

with open('./hmm_models_66_percent','rb') as file: 
    model = pickle.load(file)